In [ ]:
import numpy as np
import pandas as pd

In [ ]:
tags= pd.read_csv('tags_output2.csv')

In [ ]:
movie_rat= pd.read_csv('movie_rating.csv')

In [ ]:
movie_rat.sample(3)

,movieId,title,Year,rating,n_of_ratings
8196,103235,"Best Offer, The",2013,4.000000,5
3032,4056,"Pledge, The",2001,3.285714,7
897,1194,Cheech and Chong's Up in Smoke,1978,3.590909,11


In [ ]:
tags= tags.merge(movie_rat, how='left', on='movieId')

In [ ]:
tags= tags[['userId', 'movieId','rating', 'n_of_ratings', 'tag', 'tag_tokens', 'tag_nsw', 'tag_lemm',
       'tag_pos_tags']]

In [ ]:
tags.sample(5)

,userId,movieId,rating,n_of_ratings,tag,tag_tokens,tag_nsw,tag_lemm,tag_pos_tags
2897,567,541,4.100806,124,philosophical,['philosophical'],['philosoph'],['philosophical'],"[('philosophical', 'JJ')]"
3209,567,109487,3.993151,73,visually appealing,"['visually', 'appealing']","['visual', 'appeal']","['visually', 'appealing']","[('visually', 'RB'), ('appealing', 'VBG')]"
1934,474,4638,2.847222,36,dinosaurs,['dinosaurs'],['dinosaur'],['dinosaur'],"[('dinosaurs', 'NNS')]"
668,357,7078,3.833333,3,bette davis,"['bette', 'davis']","['bett', 'davi']","['bette', 'davis']","[('bette', 'NN'), ('davis', 'NN')]"
1929,474,4558,2.464286,14,twins,['twins'],['twin'],['twin'],"[('twins', 'NNS')]"


In [ ]:
tags['high_rating'] = (tags['rating'] >= 3).astype(int)


tags.sample(5)

,userId,movieId,rating,n_of_ratings,tag,tag_tokens,tag_nsw,tag_lemm,tag_pos_tags,high_rating
2598,477,4725,3.625000,4,well done,"['well', 'done']","['well', 'done']","['well', 'done']","[('well', 'RB'), ('done', 'VBN')]",1
937,424,81591,3.630952,42,atmospheric,['atmospheric'],['atmospher'],['atmospheric'],"[('atmospheric', 'JJ')]",1
13,18,1221,4.259690,129,al pacino,"['al', 'pacino']","['al', 'pacino']","['al', 'pacino']","[('al', 'NN'), ('pacino', 'NN')]",1
3117,567,48780,4.005556,90,atmospheric,['atmospheric'],['atmospher'],['atmospheric'],"[('atmospheric', 'JJ')]",1
1424,474,1363,3.500000,5,religion,['religion'],['religion'],['religion'],"[('religion', 'NN')]",1


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
ratings=tags['high_rating']

In [ ]:
pip install imbalanced-learn

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

train_tags, test_tags, train_ratings, test_ratings = train_test_split(
    tags['tag_nsw'], ratings, test_size=0.2, random_state=42
)

print("Before Oversampling - Training Set Class Distribution:\n", train_ratings.value_counts(normalize=True))

oversampler = RandomOverSampler(random_state=42)

train_tags_resampled, train_ratings_resampled = oversampler.fit_resample(train_tags.values.reshape(-1, 1), train_ratings)

print("\nAfter Oversampling - Training Set Class Distribution:\n", train_ratings_resampled.value_counts(normalize=True))

train_tags_resampled = pd.DataFrame(train_tags_resampled, columns=['tag_nsw'])


Before Oversampling - Training Set Class Distribution:
 1    0.92091
0    0.07909
Name: high_rating, dtype: float64

After Oversampling - Training Set Class Distribution:
 1    0.5
0    0.5
Name: high_rating, dtype: float64


In [ ]:
 # TF-IDF vectorization
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = vectorizer.fit_transform(train_tags_resampled['tag_nsw'])
X_test_tfidf = vectorizer.transform(test_tags)



Logistic Regression:

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr_model = LogisticRegression()
lr_model.fit(X_train_tfidf, train_ratings_resampled)

y_pred_lr = lr_model.predict(X_test_tfidf)

accuracy_lr = accuracy_score(test_ratings, y_pred_lr)
print(f"Accuracy Logistic Regression: {accuracy_lr}")

Accuracy Logistic Regression: 0.8073270013568521


Support Vector Machines (SVM):

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

svm_model = SVC(kernel='linear')
svm_model.fit(X_train_tfidf, train_ratings_resampled)

y_pred_svm = svm_model.predict(X_test_tfidf)

accuracy_svm = accuracy_score(test_ratings, y_pred_svm)
print(f"Accuracy SVM: {accuracy_svm}")

Accuracy SVM: 0.8005427408412483


Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf_model = RandomForestClassifier()
rf_model.fit(X_train_tfidf, train_ratings_resampled)

y_pred_rf = rf_model.predict(X_test_tfidf)

accuracy_rf = accuracy_score(test_ratings, y_pred_rf)
print(f"Accuracy Random Forest: {accuracy_rf}")

Accuracy Random Forest: 0.8059701492537313


Gradient Boosting (XGBoost):

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

xgb_model = XGBClassifier()
xgb_model.fit(X_train_tfidf, train_ratings_resampled)

y_pred_xgb = xgb_model.predict(X_test_tfidf)

accuracy_xgb = accuracy_score(test_ratings, y_pred_xgb)
print(f"Accuracy XGBoost: {accuracy_xgb}")

Accuracy XGBoost: 0.8765264586160109
